# 2501 Lecture - Data Wrangling / Visualization


### Ryan Kazmerik
* Data Scientist, Encana Corporation
* Sessional Instructor, Mount Royal University
* Mount Royal University, Bachelor CIS (2011)
* Wilfrid Laurier University, Master MAC (2019)

In [ ]:
# pip install pandas
# pip install altair
# pip install spacy
# python -m spacy download en_core_web_sm

## Let's start with our first data representation: Row based files

## For this we'll load up our data in CSV format, and use the built in Python library CSV to read the contents of the file:

In [ ]:
import csv

with open('articles.csv',  encoding="utf8") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    
    for row in csv_reader:
        print(", ".join(row), end='\n\n')


## CSV is a great storage format, compact, and readable - but a little clumsy to work with.

## Let's convert this CSV into our 2nd data structure: List

In [ ]:
articles_list = []

with open('articles.csv',  encoding="utf8") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    
    for row in csv_reader:
        articles_list.append(row)
    

## Let's see how many articles are in our list (a.k.a length of the list), and how many columns of metadata each article has (a.k.a length of the first item):

In [ ]:
total_articles = #TODO: GET LENGTH OF THE LIST 
total_columns = #TODO: GET LENGTH OF FIRST ITEM IN LIST

print('Total number of articles:', total_articles) #a.k.a the length of the list

print('Total number of columns:', total_columns) #a.k.a the length of the first object in the list

## A list wouldn't be a list if we couldn't iterate through the items.

## Let's use Pythons built in range function [0:9] to print the titles of the first 10 articles:

In [ ]:
for article in articles_list[0:10]: #not ideal that the first item is always the column name
    
    print(article[4], end="\n\n") 

## With a list, we can easily get some basic stats on the articles and iterate through the items.

## But if we want to add a new property to each item, lists can be difficult to work with.. so it's best to convert our list items into our 3rd data structure : Dictionary

## Let's convert the original CSV to a Dictionary this time, and print the first 10 articles

In [ ]:
articles_dict = [a for a in csv.DictReader(open('articles.csv', encoding="utf8"))]

for article in articles_dict[0:10]:

    #QUESTION: WHY IS THIS METHOD OF REFERENCING THE COLUMN BETTER THAN THE LIST METHOD?
    print(article['title'], end="\n\n") 

## Now let's add our new property to each article (word count of article)

In [ ]:
for article in articles_dict:
    
    words = # TODO: create a list of words using the split() function
    word_count = # TODO: get the length of the words list
    
    article['word_count'] = word_count
    
print(article[0])

## Working with Dictionaries can accomplish this task, but compiling aggregations (groupings) is a bit tricky.

## Let's convert our dataset into our 4th data structure : Data Frame 

## We can use the popular library Pandas to convert straight from our Dictionary to a DataFrame:

In [ ]:
import pandas as pd
from pandas import json_normalize

df = pd.DataFrame.from_dict(json_normalize(articles_dict), orient='columns')

df.info() #displays some basic info on the dataframe
df.head() #prints out the first 5 items

## Now we can easily produce some aggregations like: top 10 sources

In [ ]:
df_sources = df.groupby(['source']).agg({ #QUESTION: WHY ARE WE COPYING INTO A NEW DATA FRAME?
    'id': 'count'
}).reset_index()

df_sources.sort_values(by=['id'], inplace=True, ascending=False)

df_sources.columns = ['source', 'num_articles']

df_sources.head(10)

## It might be nice to see this data visualized, and luckily a Data Frame transposes nicely into many Python viz libraries.

## We'll use one called Altair to produce a vertical bar chart of our data:

In [ ]:
import altair as alt

alt.Chart(df_sources.head(10)).mark_bar().encode(
    x='num_articles',
    y='source'
).properties(
    title='Number of Articles by Source',
    width=750
)

## This is interesting but doesn't really tell us much about what the articles are talking about.

## Let's apply some data science using Natural Language Processing to help us to extract some more meaningful metadata from the articles.

## We can use a popular NLP library called SpaCy to do the text processing:

https://explosion.ai/demos/displacy-ent

In [ ]:
import spacy as sp
nlp = sp.load("en_core_web_sm")

In [ ]:
def extract_locations(text):

    doc = nlp(text)
    ents = [e.text for e in doc.ents if e.label_ == "LOC"]
    
    if len(ents) == 0:
        ents = None
    else:
        ents = ents[0]
        
    return ents

In [ ]:
df['locations'] = df['description'].map(extract_locations)

## Let's aggregate and visualize our new locations column to show the top 10 locations mentioned in the articles:

In [ ]:
df_locations = df.groupby(['locations']).agg({
    'id': 'count'
}).reset_index()

df_locations.sort_values(by=['id'], inplace=True, ascending=False)

df_locations.columns = ['location', 'num_articles']

df_sources.head(10)

In [ ]:
alt.Chart(df_locations.head(10)).mark_bar().encode(
    x='num_articles',
    y='location'
).properties(
    title='Number of Articles by Location',
    width=750
)

In [ ]:
#TODO: CREATE ANOTHER COLUMN IN OUR df DATAFRAME TO EXTRACT THE ORGANIZATIONS (ORG) FROM THE ARTICLES

#TODO: CREATE A NEW DATAFRAME CALLED df_organizations AND GROUP BY THE ORGANIZATION

#TODO: CREATE A NEW BAR CHART TO VISUALIZE THE ORGANIZATION DATA